In [ ]:
import pandas as pd
import numpy as np
import glob
from sklearn.linear_model import LogisticRegression
from joblib import delayed, Parallel
from tqdm import tqdm_notebook

In [ ]:
train_paths = glob.glob('./AI量化模型预测挑战赛公开数据/train/*')
len(train_paths)

In [ ]:
test_paths = glob.glob('./AI量化模型预测挑战赛公开数据/test/*')
len(test_paths)

In [116]:
df = pd.read_csv(glob.glob('./AI量化模型预测挑战赛公开数据/train/*')[1])
df = df.iloc[:, 4:]
df

,n_close,amount_delta,n_midprice,n_bid1,n_bsize1,n_bid2,n_bsize2,n_bid3,n_bsize3,n_bid4,...,n_asize3,n_ask4,n_asize4,n_ask5,n_asize5,label_5,label_10,label_20,label_40,label_60
0,0.005973,990629.0,0.006720,0.006471,1.879378e-06,0.005973,0.000025,0.005475,0.000021,0.004978,...,0.000014,0.008462,0.000016,0.008960,0.000019,1,0,1,0,0
1,0.006471,444943.0,0.006720,0.006471,8.457202e-06,0.005973,0.000027,0.005475,0.000022,0.004978,...,0.000012,0.008462,0.000016,0.008960,0.000019,0,0,1,0,0
2,0.006471,1138480.0,0.006720,0.006471,3.924584e-06,0.005973,0.000031,0.005475,0.000022,0.004978,...,0.000012,0.008462,0.000016,0.008960,0.000019,0,0,1,1,0
3,0.005973,2052770.0,0.006222,0.005973,2.631130e-05,0.005475,0.000023,0.004978,0.000011,0.004480,...,0.000007,0.007964,0.000012,0.008462,0.000016,0,0,2,0,0
4,0.006471,3249679.0,0.006222,0.005973,1.934654e-05,0.005475,0.000023,0.004978,0.000011,0.004480,...,0.000007,0.007964,0.000013,0.008462,0.000016,0,0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,-0.003982,104091.0,-0.003733,-0.003982,1.842528e-07,-0.004480,0.000007,-0.004978,0.000017,-0.005475,...,0.000024,-0.001991,0.000010,-0.001493,0.000023,1,1,1,1,2
1995,-0.003982,236168.0,-0.004231,-0.004480,7.020030e-06,-0.004978,0.000017,-0.005475,0.000016,-0.005973,...,0.000022,-0.002489,0.000024,-0.001991,0.000010,1,2,1,2,2
1996,-0.003982,20010.0,-0.004231,-0.004480,7.020030e-06,-0.004978,0.000017,-0.005475,0.000016,-0.005973,...,0.000022,-0.002489,0.000024,-0.001991,0.000011,1,2,1,2,2
1997,-0.003982,1095195.0,-0.003733,-0.003982,1.670804e-06,-0.004480,0.000007,-0.004978,0.000017,-0.005475,...,0.000024,-0.001991,0.000011,-0.001493,0.000023,1,1,1,1,1


In [117]:
df = pd.read_csv(glob.glob('./AI量化模型预测挑战赛公开数据/test/*')[1])
df = df.iloc[:, 4:]

In [118]:
df.columns

Index(['n_close', 'amount_delta', 'n_midprice', 'n_bid1', 'n_bsize1', 'n_bid2',
       'n_bsize2', 'n_bid3', 'n_bsize3', 'n_bid4', 'n_bsize4', 'n_bid5',
       'n_bsize5', 'n_ask1', 'n_asize1', 'n_ask2', 'n_asize2', 'n_ask3',
       'n_asize3', 'n_ask4', 'n_asize4', 'n_ask5', 'n_asize5'],
      dtype='object')

In [119]:
def base_feature(path, train=True):
    df_feat = []
    df = pd.read_csv(path)
    for idx, row in enumerate(df.iterrows()):
        idx_feat = []
        for col in ['n_close', 'amount_delta', 'n_midprice', 'n_bid1', 'n_bsize1', 'n_bid2',
           'n_bsize2', 'n_bid3', 'n_bsize3', 'n_bid4', 'n_bsize4', 'n_bid5',
           'n_bsize5', 'n_ask1', 'n_asize1', 'n_ask2', 'n_asize2', 'n_ask3',
           'n_asize3', 'n_ask4', 'n_asize4', 'n_ask5', 'n_asize5']:

            idx_feat.append(row[1][col])

            if idx == 0:
                idx_feat.append(np.nan)
            else:
                idx_feat.append(row[1][col] - df.iloc[max(0, idx-3):idx][col].mean())
                idx_feat.append(row[1][col] - df.iloc[max(0, idx-10):idx][col].mean())
                idx_feat.append(row[1][col] - df.iloc[max(0, idx-20):idx][col].mean())
                
                idx_feat.append(row[1][col] - df.iloc[max(0, idx-3):idx][col].max())
                idx_feat.append(row[1][col] - df.iloc[max(0, idx-10):idx][col].max())
                idx_feat.append(row[1][col] - df.iloc[max(0, idx-20):idx][col].max())

        if train:
            idx_feat += list(row[1].iloc[-5:])
        
        df_feat.append(idx_feat)
    
    return pd.DataFrame(df_feat)

In [ ]:
train_feat = Parallel(n_jobs=5)(delayed(base_feature)(path, True) for path in tqdm_notebook(train_paths[:]))

/tmp/ipykernel_16553/2475458747.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  train_feat = Parallel(n_jobs=5)(delayed(base_feature)(path, True) for path in tqdm_notebook(train_paths[:]))


  0%|          | 0/1225 [00:00<?, ?it/s]

In [76]:
test_feat = Parallel(n_jobs=5)(delayed(base_feature)(path, False) for path in tqdm_notebook(test_paths[:]))

/tmp/ipykernel_16553/1375492100.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  test_feat = Parallel(n_jobs=5)(delayed(base_feature)(path, False) for path in tqdm_notebook(test_paths[:]))


  0%|          | 0/296 [00:00<?, ?it/s]

In [77]:
train_feat = pd.concat(train_feat, 0)

/tmp/ipykernel_16553/672406464.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  train_feat = pd.concat(train_feat, 0)


In [88]:
import lightgbm as lgb
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score

val_pred = cross_val_predict(
    lgb.LGBMClassifier(),
    train_feat.iloc[:, :-5], train_feat.iloc[:, -5]
)
print(f1_score(train_feat.iloc[:, -5], val_pred, average='macro'))

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.205441 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11730
[LightGBM] [Info] Number of data points in the train set: 1959020, number of used features: 46
[LightGBM] [Info] Start training from score -1.906815
[LightGBM] [Info] Start training from score -0.353174
[LightGBM] [Info] Start training from score -1.903860
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.198348 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11730
[LightGBM] [Info] Number of data points in the train set: 1959020, number of used features: 46
[LightGBM] [Info] Start training from score -1.906815
[LightGBM] [Info] Start training from score -0.353174
[LightGBM] [Info] Start training from score -1.903860
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing w

In [ ]:
m5 = lgb.LGBMClassifier()
m5.fit(train_feat.iloc[:, :-5], train_feat.iloc[:, -5])

m10 = lgb.LGBMClassifier()
m10.fit(train_feat.iloc[:, :-5], train_feat.iloc[:, -4])

m20 = lgb.LGBMClassifier()
m20.fit(train_feat.iloc[:, :-5], train_feat.iloc[:, -3])

m40 = lgb.LGBMClassifier()
m40.fit(train_feat.iloc[:, :-5], train_feat.iloc[:, -2])

m60 = lgb.LGBMClassifier()
m60.fit(train_feat.iloc[:, :-5], train_feat.iloc[:, -1])

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.042072 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11730
[LightGBM] [Info] Number of data points in the train set: 2448775, number of used features: 46
[LightGBM] [Info] Start training from score -1.906813
[LightGBM] [Info] Start training from score -0.353175
[LightGBM] [Info] Start training from score -1.903860
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.040548 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11730
[LightGBM] [Info] Number of data points in the train set: 2448775, number of used features: 46
[LightGBM] [Info] Start training from score -1.568960
[LightGBM] [Info] Start training from score -0.534057
[LightGBM] [

In [102]:
# !mkdir submit

In [105]:
for df, path in zip(test_feat, test_paths):
    sub = pd.DataFrame({
        'uuid': range(len(df)),
        'label_5': m5.predict(df),
        'label_10': m10.predict(df),
        'label_20': m20.predict(df),
        'label_40': m40.predict(df),
        'label_60': m60.predict(df)
    })
    sub.to_csv('./submit/' + path.split('/')[-1], index=None)